In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../functions/')
from statistics_functions import *
import numpy as np

In [2]:
res = '4km'

path = '/work/bk1377/a270230/'
path_stat = path + 'statistics/'

In [3]:
# load the lkf data
data = np.load(path_stat + f'lkfs_paths_{res}.npz', allow_pickle=True)
years, lkfs, paths, paths_all = [data[key] for key in data.files]

LKFs = np.load(path_stat + f'LKFs_{res}.npy', allow_pickle=True)[0]

In [4]:
# add deformation, divergence, shear, and vorticity to the LKF dictionary
for year in years:
    
    y = np.where(years==year)[0][0]
    for d in range(365):

        eps_d, div_d, shr_d, vor_d = [], [], [], []
        for lkf in lkfs[y][d]:

            div_d += np.mean(lkf[:,4]),
            shr_d += np.mean(lkf[:,5]),
            vor_d += np.mean(lkf[:,6]),

        eps_d = np.sqrt(np.array(div_d)**2+np.array(shr_d)**2)

        LKFs[f'y{year}'][f'd{d}']['deformation'] = eps_d
        LKFs[f'y{year}'][f'd{d}']['divergence']  = div_d
        LKFs[f'y{year}'][f'd{d}']['shear']       = shr_d
        LKFs[f'y{year}'][f'd{d}']['vorticity']   = vor_d

In [5]:
# determine whether an lkf is a lead (1), a ridge (2) or not quantifiable (=longer lived) (3).
# "divergence<0 => lead" only works for the first day of the lkf. on the second day, a lead
# can have divergence>0 but is still a region of open water

for year in years:
    LKFs[f'y{year}']['d0']['lead or ridge'] = np.where(LKFs[f'y{year}']['d0']['divergence']<0, 1, 2)
    
    y = np.where(years==year)[0][0]
    for d in range(1,365):
        
        LKFs[f'y{year}'][f'd{d}']['lead or ridge'] = np.where(LKFs[f'y{year}'][f'd{d}']['divergence']<0, 1, 2)
        for lkf in range(len(lkfs[y][d])):
            
            for lkf_path in paths_all[0][0]:
                if np.size(lkf_path)>1:
                    if lkf_path[1] == lkf:
                        LKFs[f'y{year}'][f'd{d}'].loc[lkf, 'lead or ridge'] = 3
                        break

In [6]:
np.save(path_stat + f'LKFs_{res}', [LKFs])

In [7]:
LKFs['y2013']['d50']

,length,lifetime,deformation,divergence,shear,vorticity,lead or ridge,growth rate,curvature
0,155.726596,9,0.905855,0.124332,0.897282,0.428772,3,92.019678,0.060314
1,106.801510,4,0.402554,-0.257307,0.309585,0.073837,3,61.066682,0.048337
2,480.090023,1,0.070793,-0.048167,0.051881,0.005519,1,NaN,0.056667
3,659.639454,4,0.086747,0.055934,0.066306,0.002574,3,78.977619,0.065917
4,375.213003,2,0.042023,0.010567,0.040673,0.016702,3,-333.323025,0.118855
...,...,...,...,...,...,...,...,...,...
84,50.565676,1,0.563377,-0.126712,0.548943,-0.264328,1,NaN,0.142136
85,158.111444,4,0.872902,0.493917,0.719725,0.108128,2,-52.956968,0.224010
86,58.497117,1,0.211643,-0.125345,0.170532,0.126451,3,NaN,0.094824
87,75.540233,1,0.125278,0.088414,0.088756,-0.019875,2,NaN,0.022098
